In [ ]:
import glob
import os
import pyodbc
import pandas as pd
import re
import numpy as np

# DATABASE CONNECTION
    # Change database_loc According to where the pathline where the Microsoft Access Database is stored 
database_loc = "C:\\Users\\paul2\\OneDrive\\Documents\\Database_Assignement.accdb;"
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+database_loc)
cursor = conn.cursor()

# Creating Data schema for each CSV
data_schema = {'study_101': "([Respondent_ID], [Q1], [Q2], [Q3], [Q4], [Q5], [Sex],[Date])VALUES(?,?,?,?,?,?,?,?)",
'study_102':"([Respondent_ID], [Q1], [Q2], [Q3], [Q5], [Sex],[Number of children],[Date])VALUES(?,?,?,?,?,?,?,?)"}

# Gathering all CSV path line from data file
path = r'C:\Users\paul2\Desktop\Interview pack resources\data'
all_CSVs = glob.glob(os.path.join(path, "*.csv"))
# CSV Processing
# Upload the CSV to each Access database
for study in all_CSVs:
    study_data = pd.read_csv(study)
    table_name = re.search(r"data\\(.*).csv",str(study)).group(1)
    insert_comm = "INSERT INTO {} ".format(table_name)+data_schema[table_name]
    # all number will have to be change to data type Float due to error raise
    for index,row in study_data.astype('float',errors='ignore').iterrows():
        cursor.execute(insert_comm
                    , (row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7]))
        conn.commit()


In [11]:
import glob
import os
import pyodbc
import pandas as pd
import re
import numpy as np

# DATABASE CONNECTION
    # Change database_loc According to where the pathline where the Microsoft Access Database is stored 
database_loc = "C:\\Users\\paul2\\OneDrive\\Documents\\Database_Assignement.accdb;"
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+database_loc)
cursor = conn.cursor()


# Gathering all CSV path line from data file
path = r'C:\Users\paul2\Desktop\Interview pack resources\data'
all_CSVs = glob.glob(os.path.join(path, "*.csv"))

# Creating Data schema for each CSV
data_schema = {'Codebook': "([id], [question_id], [value], [label])VALUES(?,?,?,?)",
'Questions_table':"([id], [question_text])VALUES(?,?)",
'Respondents_table':"([id], [study_id], [sex], [date])VALUES(?,?,?,?)",
'Responses_table':"([id], [respondent_id], [study_id], [question_id], [response])VALUES(?,?,?,?,?)"}


cursor.execute("INSERT INTO Codebook"+data_schema['Codebook']
            , ("131","Q5","2",None))
cursor.execute("INSERT INTO Codebook"+data_schema['Codebook']
            , ("132","Number_of_children",None,None))
cursor.execute("INSERT INTO Questions_table"+data_schema['Questions_table']
            , ("Number_of_children",None))
conn.commit()

info = pd.read_csv(all_CSVs[1])
info_Respondents_table = info.drop(['Q1','Q2','Q3', 'Q5','Number of children'], axis='columns')
info_Respondents_table.insert(1, 'study_id', 102)
Respondents_table = 11
for index,row in info_Respondents_table.astype('float',errors='ignore').iterrows():
        cursor.execute("INSERT INTO Respondents_table"+data_schema['Respondents_table']
                    , (float(Respondents_table),row[1],row[2],row[3]))
        Respondents_table +=1
        conn.commit()
        
info_Responses_table = info.drop(['date'], axis='columns')
Responses_table_id = 101
for index,row in info_Responses_table.astype('float',errors='ignore').iterrows():
    for key,item in dict(row['Q1':]).items():
        if key == 'Q3':
            for x in dict(row)[key].split(';'):
                cursor.execute("INSERT INTO Responses_table"+data_schema['Responses_table']
                                , (float(Responses_table_id),row['respondent_id'],float(102),key+'_'+str(x),x))
                Responses_table_id += 1
        else:
            cursor.execute("INSERT INTO Responses_table"+data_schema['Responses_table']
                            , (float(Responses_table_id),row['respondent_id'],float(102),key,item))
            Responses_table_id += 1
    conn.commit()

2.	Using the database you created, extract information that will answer the following questions:


a.	For Q1, how many people responded ‘strongly disagree’ or ‘disagree’?

b.	For Q1, how many people of each sex responded ‘strongly disagree’ or ‘disagree’? 

c.	For Q1, how many people of each sex responded:
i.	‘strongly disagree’ or ‘disagree’;
ii.	‘neutral’;
iii.	‘strongly agree’ or ‘agree’?
The final output should be contained within one table.

d.	For Q3, what proportion of people selected each option?

e.	For Q3, what proportion of people of each sex selected each option?

f.	For respondents in study_102, how many people had (i) no children, (ii) 1-2 children, and (iii) 3 or more children? The final output should be contained within one table.


In [235]:
# Question 2a
query ='''

select count(response) as Counter
from Responses_table
where question_id = 'Q1'
and response in (1,2)
;

'''
df = pd.read_sql(query, conn)
df.to_csv("output_Task2a.csv", index=False, encoding='utf8')

In [238]:
# Question 2b
query ='''

SELECT Responses_table.respondent_id, Responses_table.study_id, Respondents_table.sex
FROM Respondents_table 
INNER JOIN Responses_table ON (Responses_table.respondent_id = Respondents_table.id) AND (Respondents_table.study_id = Responses_table.study_id)
WHERE (((Responses_table.[question_id])='Q1') AND ((Responses_table.[response]) In (1,2)));

union all
SELECT t1.respondent_id, t1.study_id, 
   t2.response as sex
FROM Responses_table t1
INNER JOIN Responses_table t2
   ON t2.respondent_id = t1.respondent_id
    and t2.study_id = t1.study_id
where 
(((t1.[question_id])='Q1') AND ((t1.[response]) In (1,2)))
and ((t2.[question_id])='sex')
;

'''
df = pd.read_sql(query, conn)
df['sex'].value_counts()
conditions = [
                (df['sex'] == 1) ,
                (df['sex'] == 2) 
             ]
choices = ['Male','Female']
df['sex'] = np.select(conditions, choices, default=None)
df = df['sex'].value_counts().to_frame().reset_index()
df.to_csv("output_Task2b.csv", index=False, encoding='utf8')

In [240]:
# Question 2c
query ='''

SELECT Responses_table.respondent_id, Responses_table.study_id,Responses_table.response, Respondents_table.sex
FROM Respondents_table 
INNER JOIN Responses_table ON (Responses_table.respondent_id = Respondents_table.id) AND (Respondents_table.study_id = Responses_table.study_id)
WHERE  (((Responses_table.[question_id])='Q1'))

union all
SELECT t1.respondent_id, t1.study_id, t1.response,
   t2.response as sex
FROM Responses_table t1
INNER JOIN Responses_table t2
   ON t2.respondent_id = t1.respondent_id
    and t2.study_id = t1.study_id
where 
(((t1.[question_id])='Q1') AND
 ((t2.[question_id])='sex'))
;

'''
df = pd.read_sql(query, conn)
df
conditions = [
                (df['response'] == 1) ,
                (df['response'] == 2) ,
                (df['response'] == 3) ,
                (df['response'] == 4) ,
                (df['response'] == 5) 
             ]
choices = ['Disagree', 'Disagree', 'Neutral', 'Agree', 'Agree']

df['response'] = np.select(conditions, choices, default=None)
df.drop(['respondent_id','study_id'], axis='columns')

Male = df[df['sex'] ==1]
Male = Male['response'].value_counts()
Male = pd.DataFrame({'Response':Male.index, 'Counter':Male.values})
Male['Sex'] = 'Male'

Female = df[df['sex'] !=1]
Female = Female['response'].value_counts()
Female = pd.DataFrame({'Response':Female.index, 'Counter':Female.values})
Female['Sex'] = 'Female'

frames = [Male, Female]
result = pd.concat(frames)
result.to_csv("output_Task2c.csv", index=False, encoding='utf8')

In [241]:
result

,Response,Counter,Sex
0,Agree,5,Male
1,Neutral,3,Male
2,Disagree,2,Male
0,Disagree,4,Female
1,Agree,4,Female
2,Neutral,1,Female


In [243]:
# Question 2d
query ='''
SELECT question_id,response
from responses_table
;

'''
df = pd.read_sql(query, conn)
Proportion = df[(df['question_id'].str.contains("Q3")) & (df['response']!= '0')]['response'].value_counts()
Cleanse_Proportion = pd.DataFrame({'Response':Proportion.index, 'Counter':Proportion.values})
Cleanse_Proportion['Counter']= Cleanse_Proportion['Counter']/sum(Proportion)
Cleanse_Proportion.to_csv("output_Task2d.csv", index=False, encoding='utf8')

In [244]:
Cleanse_Proportion

,Response,Counter
0,0,0.452632
1,1,0.252632
2,2,0.094737
3,5,0.084211
4,4,0.063158
5,3,0.042105
6,6,0.010526


In [245]:
# Question 2e
query ='''

SELECT Responses_table.respondent_id, Responses_table.study_id,Responses_table.question_id,Responses_table.response, Respondents_table.sex
FROM Respondents_table 
INNER JOIN Responses_table ON (Responses_table.respondent_id = Respondents_table.id) AND (Respondents_table.study_id = Responses_table.study_id)
where Responses_table.question_id <> 'Q1'
and Responses_table.question_id <> 'Q2'
and Responses_table.question_id <> 'Q4'
and Responses_table.question_id <> 'Q5'
and Responses_table.question_id <> 'Number of children'
and Responses_table.question_id <> 'Sex'

union all

SELECT t1.respondent_id, t1.study_id,t1.question_id, t1.response,
   t2.response as sex
FROM Responses_table t1
INNER JOIN Responses_table t2
   ON t2.respondent_id = t1.respondent_id
    and t2.study_id = t1.study_id
where 
t1.question_id <> 'Q1'
and t1.question_id <> 'Q2'
and t1.question_id <> 'Q4'
and t1.question_id <> 'Q5'
and t1.question_id <> 'Number of children'
and t1.question_id <> 'Sex'
AND ((t2.[question_id])='sex')
'''
df = pd.read_sql(query, conn)
Male_Proportion = df[(df['response']!='0')& (df['sex']== 1) ]['response'].value_counts()
Cleanse_Male_Proportion = pd.DataFrame({'Response':Male_Proportion.index, 'Counter':Male_Proportion.values})
Cleanse_Male_Proportion['Counter']= Cleanse_Male_Proportion['Counter']/sum(Male_Proportion)
Cleanse_Male_Proportion['Sex'] = 'Male'

Female_Proportion = df[(df['response']!='0')& (df['sex']!= 1) ]['response'].value_counts()
Cleanse_Female_Proportion = pd.DataFrame({'Response':Female_Proportion.index, 'Counter':Female_Proportion.values})
Cleanse_Female_Proportion['Counter']= Cleanse_Female_Proportion['Counter']/sum(Female_Proportion)
Cleanse_Female_Proportion['Sex'] = 'Female'

frames = [Cleanse_Male_Proportion, Cleanse_Female_Proportion]
result = pd.concat(frames)

result.to_csv("output_Task2e.csv", index=False, encoding='utf8')

In [246]:
# Question 2F
query ='''
SELECT  response
FROM Responses_table
where question_id = 'Number of children' 
;

'''
df = pd.read_sql(query, conn)
conditions = [  (df['response'] == 0) ,
            (df['response'] >=1)  & (df['response'] <=2),
            (df['response'] >= 3) 
             ]
choices = ['no children', '1-2 children', '3 or more children']

df['response'] = np.select(conditions, choices, default=None)
df = df['response'].value_counts().to_frame().reset_index()
df.to_csv("output_Task2f.csv", index=False, encoding='utf8')

In [ ]:
# help for doing conditions

In [ ]:
# Question 2d
query ='''
SELECT  Q3

FROM study_101

UNION ALL

SELECT Q3

FROM study_102
;

'''
df = pd.read_sql(query, conn)
df
counter = []
# Cleansing and Seperating all Opinion given per row then load to a list
for x in range(len(df.get_values())):
    counter.extend(df.get_values()[x][0].replace(" ","").split(';'))

all_entered_opinion =  pd.DataFrame(counter,columns=['Proportion'])['Proportion'].value_counts()
cleanse_opinion = pd.DataFrame({'Opinion':all_entered_opinion.index, 'Proportion':all_entered_opinion.values})

# Creating conditional if statement to input appropriate value in the new column 
conditions = [
                (cleanse_opinion['Opinion'] == '1') ,
                (cleanse_opinion['Opinion'] == '2') ,
                (cleanse_opinion['Opinion'] == '3') ,
                (cleanse_opinion['Opinion'] == '4') ,
                (cleanse_opinion['Opinion'] == '5') ,
                (cleanse_opinion['Opinion'] == '6') ,
             ]
choices = ['Orange', 'Blue', 'Red', 'Purple', 'Pink', 'Don’t know / Not sure']

cleanse_opinion['Opinion'] = np.select(conditions, choices, default=None)
cleanse_opinion['Proportion'] = cleanse_opinion['Proportion'] / len(df.get_values())
cleanse_opinion

In [ ]:
# Question 2e

choices = ['Orange', 'Blue', 'Red', 'Purple', 'Pink', 'Don’t know / Not sure'] # Global Variable

dataframes_list = []
# Creating a loop that creates a seperate dataframe for each Gender on their proportion each opinion
for x,y in [(1,'Male'),(2,'Female')]:
    query ='''
    Select Q3
    from(
    SELECT  Q3,Sex

    FROM study_101

    UNION ALL

    SELECT Q3 , Sex

    FROM study_102)
    Where Sex = {}
    ;

    '''.format(x)
    df = pd.read_sql(query, conn)
    df
    counter = []
    # Cleansing and Seperating all Opinion given per row then load to a list
    for x in range(len(df.get_values())):
        counter.extend(df.get_values()[x][0].replace(" ","").split(';'))
    propotion_column_header = 'Proportion({})'.format(y)
    # Counting how many people are exist in their respective Group
    all_entered_opinion =  pd.DataFrame(counter,columns=[propotion_column_header])[propotion_column_header].value_counts()
    cleanse_opinion = pd.DataFrame({'Opinion':all_entered_opinion.index, propotion_column_header :all_entered_opinion.values})
    conditions = [
                (cleanse_opinion['Opinion'] == '1') ,
                (cleanse_opinion['Opinion'] == '2') ,
                (cleanse_opinion['Opinion'] == '3') ,
                (cleanse_opinion['Opinion'] == '4') ,
                (cleanse_opinion['Opinion'] == '5') ,
                (cleanse_opinion['Opinion'] == '6') ,
             ]
    # Changing the Opinion Choices from to Number to its appropriate value
    cleanse_opinion['Opinion'] = np.select(conditions, choices, default=None)
    cleanse_opinion[propotion_column_header] = cleanse_opinion[propotion_column_header] / len(df.get_values())
    dataframes_list .append(cleanse_opinion)
    
# Then combine all dataframe into one to create an overview dataframe 
final_table = pd.DataFrame(choices,columns=['Opinion'])
for x in dataframes_list:
    final_table = final_table.merge(x)
final_table

In [ ]:
# Question 2F
query ='''
SELECT  [Number of children]

FROM study_102
;

'''
df = pd.read_sql(query, conn)
# Creating a conditional if statement to input appropriate value in the new column
conditions = [
            (df['Number of children'] == 0) ,
            (df['Number of children'] >=1)  & (df['Number of children'] <=2),
            (df['Number of children'] >= 3) ,
         ]
choices = ['no children', '1-2 children', '3 or more children']
df['Group'] = np.select(conditions, choices, default=None)
# Counting how many people are exist in their respective Group
Final_Table = pd.DataFrame({'Group':df['Group'].value_counts().index, 'Total' :df['Group'].value_counts(sort=False).values})
Final_Table

In [ ]:
import glob
import os
import pyodbc
import pandas as pd
import re
import numpy as np

# DATABASE CONNECTION
    # Change database_loc According to where the pathline where the Microsoft Access Database is stored 
database_loc = "C:\\Users\\paul2\\OneDrive\\Documents\\Database_Assignement.accdb;"
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+database_loc)
cursor = conn.cursor()


In [ ]:
query ='''

SELECT *
from Codebook
;

'''
df = pd.read_sql(query, conn)
df

In [ ]:
list(df)

In [ ]:
list(df['question_id'].unique())

In [ ]:
list(df[df['question_id'] == 'Q5']['value'])

In [ ]:
# Gathering all CSV path line from data file
path = r'C:\Users\paul2\Desktop\Interview pack resources\data'
all_CSVs = glob.glob(os.path.join(path, "*.csv"))

In [ ]:
all_CSVs[0]

In [ ]:
study_data = pd.read_csv(all_CSVs[0])

In [ ]:
list(study_data['Q5'].unique())

In [ ]:
for x in list(study_data['Q5'].unique()):
    for x not in list(df[df['question_id'] == 'Q5']['value']):
        print(x)

In [ ]:
for x in list(study_data['Q5'].unique()):
    if x not in list(df[df['question_id'] == 'Q5']['value']):
        print(x)
    

In [ ]:
# Creating Data schema for each CSV
data_schema = {'study_101': "([Respondent_ID], [Q1], [Q2], [Q3], [Q4], [Q5], [Sex],[Date])VALUES(?,?,?,?,?,?,?,?)",
'study_102':"([Respondent_ID], [Q1], [Q2], [Q3], [Q5], [Sex],[Number of children],[Date])VALUES(?,?,?,?,?,?,?,?)"}

insert_comm = "INSERT INTO {} ".format(table_name)+data_schema[table_name]
# all number will have to be change to data type Float due to error raise
for index,row in study_data.astype('float',errors='ignore').iterrows():
    cursor.execute(insert_comm
                , (row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7]))
    conn.commit()

In [ ]:
data_schema = {'study_101': "([Respondent_ID], [Q1], [Q2], [Q3], [Q4], [Q5], [Sex],[Date])VALUES(?,?,?,?,?,?,?,?)",
'study_102':"([Respondent_ID], [Q1], [Q2], [Q3], [Q5], [Sex],[Number of children],[Date])VALUES(?,?,?,?,?,?,?,?)"}

insert_comm = "INSERT INTO {} ".format('study_101')+data_schema['study_101']

In [ ]:
dader = '?'*7

In [ ]:
','.join('?'*7)

In [ ]:
'VALUES({})'.format(','.join('?'*7))

In [ ]:
y = ''
for x in list(df):
    y+= '[{}] '.format(x)
y.strip().replace(' ',',')

In [ ]:
kl=pd.read_csv('C:\\Users\\paul2\\Desktop\\Interview pack resources\\data\\study_101.csv')

In [ ]:
for x in kl['Q5'].unique():
    print(x)

In [ ]:
# loop to add infor in codebook
for x in kl['Q5'].unique():
    if x not in df[df['question_id'] == 'Q5']['value'].unique():
        print(x)

In [ ]:
df[df['question_id'] == 'Q5']['value'].unique()

In [ ]:
list(kl)

In [ ]:
df